In [1]:
import pandas as pd
import pickle
from helper_classes.semrush.semrush_api import SEMRushAPI

# your SEMRush API Key
api_key = ''
# SEMRush Keyword Database used (Complete list https://www.semrush.com/api-analytics/)
database = 'de'

sr = SEMRushAPI(api_key, database=database)
reg_score = []

In [2]:
# Define Domain to optimize & competitors (more competitors=better results)
main_domain = ['www.barf-alarm.de']
competitor_domains = ['barf-factory.de', 'www.barfers-wellfood.de',
                      'www.barfinfo.de', 'www.barfkoeln.de', 'www.barfshop-koeln.de',
                      'www.barfshop.de']
base_domains = main_domain + competitor_domains

In [4]:
# Get Information about the Domains (Quelle: SEMRUSH)
domain_arr = []
for domain in base_domains:
    sr_result = sr.get_domain_overview(domain)
    if sr_result:
        domain_arr.append(sr_result[0])

In [5]:
domain_df = pd.DataFrame.from_records(domain_arr)

In [ ]:
domain_df

In [ ]:
# Get all Keywords for the listed Domains (Quelle: SEMRUSH API)
# This can take a while an need API Credits
# The results of the API will be cached
keyword_df = None
keyword_df = pd.DataFrame()
for domain in base_domains:
    res = sr.get_domain_keywords(domain, count=2000)
    if res:
        kw_df = pd.DataFrame.from_records(res)
        kw_df["Domain"] = domain
        keyword_df = pd.concat([kw_df, keyword_df])

In [8]:
# Delete not needed columns
if "Position Difference" in keyword_df:
    del keyword_df["Position Difference"]
if "Previous Position" in keyword_df:
    del keyword_df["Previous Position"]
if "Trends" in keyword_df:
    del keyword_df["Trends"]
if "Traffic (%)" in keyword_df:
    del keyword_df["Traffic (%)"]
if "Traffic Cost (%)" in keyword_df:
    del keyword_df["Traffic Cost (%)"]

# Convert Strings to numbers
keyword_df["CPC"] = pd.to_numeric(keyword_df["CPC"]).astype(float)
keyword_df["Competition"] = pd.to_numeric(keyword_df["Competition"]).astype(float)
keyword_df["Search Volume"] = pd.to_numeric(keyword_df["Search Volume"]).astype(float)
keyword_df["Position"] = pd.to_numeric(keyword_df["Position"]).astype(float)
keyword_df["Number of Results"] = pd.to_numeric(keyword_df["Number of Results"]).astype(float)

In [9]:
keyword_df.sort_values('Search Volume', ascending=False).head(10)

,CPC,Competition,Keyword,Number of Results,Position,Search Volume,Url,Domain
26,0.77,0.01,kreuzberg,15700000.0,24.0,74000.0,https://www.barfers-wellfood.de/barf-berlin-kr...,www.barfers-wellfood.de
924,1.11,1.00,kokosöl,3080000.0,99.0,49500.0,https://www.barf-alarm.de/blog/kokosoel-das-na...,www.barf-alarm.de
1411,1.11,1.00,kokosöl,3080000.0,62.0,49500.0,https://www.barfers-wellfood.de/lunderland/lun...,www.barfers-wellfood.de
126,1.57,0.65,durchfall,6520000.0,41.0,49500.0,https://www.barfers-wellfood.de/durchfall-hund,www.barfers-wellfood.de
3,1.04,1.00,msm,45500000.0,53.0,33100.0,http://barfshop.de/,www.barfshop.de
717,0.93,0.05,frettchen,1180000.0,77.0,22200.0,http://www.barfinfo.de/barf-fuer-frettchen/,www.barfinfo.de
1,1.43,0.89,barfen,394000.0,4.0,14800.0,https://www.barfers-wellfood.de/barf-barfen,www.barfers-wellfood.de
82,1.43,0.89,barfen,394000.0,59.0,14800.0,https://www.barf-alarm.de/blog/barf-bei-aelter...,www.barf-alarm.de
2,1.43,0.89,barfen,394000.0,3.0,14800.0,http://www.barfinfo.de/was-ist-barfen/,www.barfinfo.de
0,1.43,0.89,barfen,394000.0,2.0,14800.0,http://www.barfinfo.de/,www.barfinfo.de


In [10]:
# Each keyword only once
keyword_df_unique = keyword_df.drop_duplicates(subset='Keyword', keep="last")

# Delete not neded rows
del keyword_df_unique["Position"]
del keyword_df_unique["Url"]
del keyword_df_unique["Domain"]

print("Count unique Keywords for all Domains: {}".format(len(keyword_df_unique.index)))
keyword_df_unique.head(10)

Count unique Keywords for all Domains: 3674


,CPC,Competition,Keyword,Number of Results,Search Volume
2,0.84,1.00,flor essence,410000.0,2400.0
3,1.04,1.00,msm,45500000.0,33100.0
4,0.80,1.00,katzenkralle,372000.0,2400.0
7,1.18,0.90,kolloidalsilber,7000.0,30.0
8,0.56,1.00,colostrum dr wolz,239000.0,40.0
10,0.34,1.00,colostrum immun,85000.0,50.0
11,0.63,1.00,dr wolz colostrum,28000.0,50.0
13,0.52,1.00,dr wolz darmflora plus,138000.0,3600.0
14,0.54,0.89,barf kräuter,110000.0,50.0
15,1.27,1.00,colostrum katze,187000.0,50.0


-------------

In [11]:
# Get Information about the found urls of the positions
# Reduce to only competitors
all_urls = keyword_df.drop_duplicates(subset='Url', keep="last")

urls_to_crawl_df = all_urls[all_urls['Domain'].isin(base_domains)]
urls_to_crawl = urls_to_crawl_df.Url
print("Count of URLs to crawl: {}".format(len(urls_to_crawl)))

Count of URLs to crawl: 400


In [12]:
# load a simple crawler to fetch URLs
from helper_classes.simple_crawler.simple_crawler import Crawler
crawler = Crawler(30) # How much Threads in parallel?

# Crawl
# Crawl results are cached
res = crawler.get_websites(urls_to_crawl)

# Build DF from cralwed URLS
crawled_df = pd.DataFrame.from_records(res)
crawled_df.rename(columns={'url': 'Url'}, inplace=True)

Processed 400 URLs


In [13]:
print("Title & Content for {} Urls".format(len(crawled_df.index)))
crawled_df.head()

Title & Content for 400 Urls


,response_content,response_status,title,Url
0,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Kolloidalsilber,https://www.barfshop.de/de/category-3/Kolloida...
1,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - BARF SENIOR - im Alter & bei Erk...,https://www.barfshop.de/de/barf-senior-im-alte...
2,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - DHN® Katzenkralle 100 g fein gem...,https://www.barfshop.de/de/dhn-katzenkralle-10...
3,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - DHN® BIO Hanföl* 250 ml,https://www.barfshop.de/de/dhn-bio-hanfoel-250...
4,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Das BARF-Buch: Inklusive 14 Reze...,https://www.barfshop.de/de/das-barf-buch-inklu...


In [14]:
# Combine Keywords and positions with crawled content
combined_df = pd.merge(keyword_df, crawled_df, on='Url', how='outer')

In [15]:
combined_df.head()

,CPC,Competition,Keyword,Number of Results,Position,Search Volume,Url,Domain,response_content,response_status,title
0,1.75,0.92,barf shop,8180000.0,2.0,5400.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite
1,1.54,0.88,barfer shop,173000.0,2.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite
2,1.31,0.74,barf shop hamburg,91000.0,14.0,480.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite
3,1.37,0.80,barfshops,56000.0,7.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite
4,1.16,0.87,barf.de,10200000.0,30.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite


In [16]:
# Check each Title, URL, Content if words of Keyword phrase are in it and save information as float
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from urllib.parse import urlparse
stopwords_list = set(stopwords.words('german'))

def check_keyword(args):
    
    words = args["Keyword"].split(" ")
    words_wo_stop = [i for i in words if not i in stopwords_list]
    count_kw = len(words_wo_stop)
    
    found_words_title = 0
    found_words_content = 0
    found_words_url = 0
    found_words_domain = 0
    url_found = 0.0
    content_found = 0.0
    title_found = 0.0
    domain_found = 0.0
    is_homepage = 0
    for word in words_wo_stop:
        if str(word) in str(args["Domain"]):
            found_words_domain += 1 

        path = urlparse(args["Url"]).path
        if str(word) in str(path):
            found_words_url += 1 
        if "/" == path:
            is_homepage = 1
        if str(word) in str(args["title"]):
            found_words_title += 1
        if str(word) in str(args["response_content"]):
            found_words_content += 1
    if found_words_url > 0:
        url_found = found_words_url / count_kw
    if found_words_title > 0:
        title_found = found_words_title / count_kw
    if found_words_content > 0:
        content_found = found_words_content / count_kw
    if found_words_content > 0:
        domain_found = found_words_domain / count_kw
    return pd.Series({'title_found': title_found, 
                      'content_found': content_found, 
                      'url_found': url_found, 
                      'domain_found': domain_found,
                      'is_homepage': is_homepage})
rated_df = combined_df.join(combined_df.apply(check_keyword, axis=1))


In [17]:
rated_df.head()

,CPC,Competition,Keyword,Number of Results,Position,Search Volume,Url,Domain,response_content,response_status,title,content_found,domain_found,is_homepage,title_found,url_found
0,1.75,0.92,barf shop,8180000.0,2.0,5400.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.500000,1.000000,1.0,1.000000,0.0
1,1.54,0.88,barfer shop,173000.0,2.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.500000,0.500000,1.0,0.500000,0.0
2,1.31,0.74,barf shop hamburg,91000.0,14.0,480.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.333333,0.666667,1.0,0.666667,0.0
3,1.37,0.80,barfshops,56000.0,7.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.000000,0.000000,1.0,0.000000,0.0
4,1.16,0.87,barf.de,10200000.0,30.0,50.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.000000,0.000000,1.0,0.000000,0.0


In [18]:
# Set an ID for each domain and keyword so we can provide the ML Algo the ID and have a representating array
domains = list(rated_df.Domain.unique())
keywords = list(rated_df.Keyword.unique())
def get_domain_id(domain):
    return domains.index(domain)
def get_keyword_id(keyword):
    return keywords.index(keyword)
rated_df['Domain_id'] = rated_df['Domain'].apply(get_domain_id)
rated_df['Keyword_id'] = rated_df['Keyword'].apply(get_keyword_id)

In [19]:
# Schow example entrys for Keyword ID 2
rated_df[rated_df["Keyword_id"]==2].head()

,CPC,Competition,Keyword,Number of Results,Position,Search Volume,Url,Domain,response_content,response_status,title,content_found,domain_found,is_homepage,title_found,url_found,Domain_id,Keyword_id
2,1.31,0.74,barf shop hamburg,91000.0,14.0,480.0,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.333333,0.666667,1.0,0.666667,0.0,0,2
1999,1.31,0.74,barf shop hamburg,91000.0,18.0,480.0,https://www.barfers-wellfood.de/,www.barfers-wellfood.de,\nJavaScript scheint in Ihrem Browser deaktivi...,200,BARF Shop - Frischfleisch für Hunde - Barfer’...,0.666667,0.333333,1.0,0.000000,0.0,4,2


In [20]:
# get only the top 40 results
# rated_df_top40 = rated_df[rated_df["Position"]<40]

In [21]:
# Delete all rows with no response_status (if any)
rated_df.dropna(subset=['response_status'], how='all', inplace=True)
print("{} rows for machine learning".format(len(rated_df.index)))

4742 rows for machine learning


In [22]:
# Define Features (which rows should the ml algorithm should consider for learning)
features = ["Domain_id", 
            "content_found", 
            "domain_found",
            "title_found", 
            "url_found", 
            "Keyword_id", 
            "CPC", 
            "Number of Results",
            "Search Volume"]
# Define Target (What should be predicted)
target = "Position"

In [23]:
# Split in train and test data
train = rated_df.sample(frac=0.8)
test = rated_df.loc[~rated_df.index.isin(train.index)]
print ("Train rows: {}".format(len(train.index)))
print ("Test rows: {}".format(len(test.index)))

Train rows: 3794
Test rows: 948


In [24]:
# Train Model with Data
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import LeaveOneOut
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

sc = StandardScaler()

mlc = MLPClassifier(activation = 'relu', random_state=1,nesterovs_momentum=True)
loo = LeaveOneOut()
pipe = make_pipeline(sc, mlc)

# Train the Model and check wich of the Parameters works best
parameters = { "mlpclassifier__hidden_layer_sizes":[(300,),(500,)],  
              "mlpclassifier__solver" : ( "sgd", "lbfgs"), 
              "mlpclassifier__max_iter": [500, 1000, 2000], 
              "mlpclassifier__learning_rate_init":[0.001, 0.1]  }
MLPClassifierModel = GridSearchCV(pipe, parameters,n_jobs= -1,cv = 5)
MLPClassifierModel.fit(train[features], train[target])

# Save Model to file to used it later
file = open("k_t_o_MLPClassifierModel_10_comp.pkl", 'wb')
pickle.dump(MLPClassifierModel, file)
file.close()

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer

In [25]:
# Open Model from File (could be skipped if you've trained it just a second before ;) )
file = open("k_t_o_MLPClassifierModel_10_comp.pkl", 'rb')
MLPClassifierModel = pickle.load(file)
file.close()

In [26]:
rated_df[rated_df["Domain"]==main_domain]

,CPC,Competition,Keyword,Number of Results,Position,Search Volume,Url,Domain,response_content,response_status,title,content_found,domain_found,is_homepage,title_found,url_found,Domain_id,Keyword_id
3522,0.59,1.00,grünlippmuschelpulver,63000.0,1.0,1300.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2920
3523,1.21,1.00,grünlippmuschelpulver pferd,159000.0,67.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2328
3524,0.77,0.93,grünlippmuschel kapseln nebenwirkungen,23000.0,42.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2921
3525,1.24,1.00,grünlippmuschelextrakt für hunde,105000.0,45.0,210.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2323
3526,1.04,1.00,grünlippmuschelextrakt hund,51000.0,44.0,390.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2319
3527,1.08,1.00,grünlippmuschel pulver,47000.0,19.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.500000,0.0,0.0,0.5,0.500000,5,2922
3528,1.14,0.95,grünlippmuschelextrakt nebenwirkungen,7000.0,30.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2923
3529,0.26,0.90,nebenwirkungen grünlippmuschel kapseln,23000.0,33.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2924
3530,1.17,1.00,grünlippmuschelpulver für hunde,59000.0,15.0,40.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2326
3531,1.30,1.00,neuseeländische grünlippmuschel für hunde,28000.0,34.0,90.0,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wünschen Sie einen individuellen Rinderfleis...,200,Grünlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.0,0.0,0.0,0.000000,5,2325


In [27]:
# Get Data to query the Model
id = 3538 # just change ID to one of the row ids from above
data = [
    rated_df.ix[id]["Domain_id"],
    rated_df.ix[id]["content_found"],
    rated_df.ix[id]["domain_found"],
    rated_df.ix[id]["title_found"],
    rated_df.ix[id]["url_found"],
    rated_df.ix[id]["Keyword_id"],
    rated_df.ix[id]["CPC"],
    rated_df.ix[id]["Number of Results"],
    rated_df.ix[id]["Search Volume"],
]

In [28]:
# What Data we query?
def print_data(data):
    print("Domain: {}".format(domains[int(data[0])]))
    print("Keyword: {}".format(keywords[int(data[5])]))
    print("Keyword in Content Found? {}".format(data[1]))
    print("Keyword in Title Found? {}".format(data[3]))
    print("Keyword in Domain Found? {}".format(data[2]))
    print("Keyword in URL Found? {}".format(data[4]))
    print("CPC: {}".format(data[6]))
    print("Number of Results: {}".format(data[7]))
    print("Search Volume: {}".format(data[8]))
print_data(data)

Domain: www.barf-alarm.de
Keyword: rohfleisch hund
Keyword in Content Found? 0.5
Keyword in Title Found? 0.0
Keyword in Domain Found? 0.0
Keyword in URL Found? 0.5
CPC: 1.21
Number of Results: 32000.0
Search Volume: 90.0


In [29]:
# Predict Position
df_to_predict = pd.DataFrame(data = [data], index=[0], columns=features)
res = MLPClassifierModel.predict(df_to_predict)
print("MLPClassifierModel predicted Position:  {}".format(int(res[0])))

MLPClassifierModel predicted Position:  13


In [30]:
# Modify Data for that Keyword
modified_data = data
data[1] = 1
data[3] = 1
print_data(modified_data)

Domain: www.barf-alarm.de
Keyword: rohfleisch hund
Keyword in Content Found? 1
Keyword in Title Found? 1
Keyword in Domain Found? 0.0
Keyword in URL Found? 0.5
CPC: 1.21
Number of Results: 32000.0
Search Volume: 90.0


In [31]:
# Predict Position
df_to_predict = pd.DataFrame(data = [modified_data], index=[0], columns=features)
res = MLPClassifierModel.predict(df_to_predict)
print("MLPClassifierModel predicted Position:  {}".format(int(res[0])))

MLPClassifierModel predicted Position:  2


In [32]:
# Now we want to predict Positions for all Keywords if title & Description are optimized for one of the Domains

In [33]:
# Get a list with all Keywords and if any with position and found Url
predict_df = pd.merge(keyword_df[keyword_df["Domain"]==main_domain][["Position", "Keyword", "Url"]],keyword_df_unique, on='Keyword', how='outer')

In [34]:
predict_df.head()

,Position,Keyword,Url,CPC,Competition,Number of Results,Search Volume
0,1.0,grünlippmuschelpulver,https://www.barf-alarm.de/shop/barf-nahrungser...,0.59,1.00,63000.0,1300.0
1,11.0,fleisch auftauen,https://www.barf-alarm.de/blog/einfrieren-und-...,2.32,0.08,395000.0,1600.0
2,2.0,markknochen hund,https://www.barf-alarm.de/blog/beinscheiben-ve...,0.56,0.29,53000.0,390.0
3,5.0,kokosöl für hunde,https://www.barf-alarm.de/blog/kokosoel-das-na...,0.79,1.00,388000.0,720.0
4,7.0,barf fleisch,https://www.barf-alarm.de/,1.42,0.93,273000.0,880.0


In [35]:
domain_id = get_domain_id(main_domain)
# Predict Data for each row
def optimized_position(args):
    try: 
        keyword_id = get_keyword_id(args["Keyword"])
    except:
        return pd.Series({'optimized_position': None })
    data = [
        domain_id,
        1,
        0,
        1,
        0,
        keyword_id,
        args["CPC"],
        args["Number of Results"],
        args["Search Volume"]
    ]
    df_to_predict = pd.DataFrame(data = [data], index=[0], columns=features)
    res = MLPClassifierModel.predict(df_to_predict)
    return pd.Series({'optimized_position': int(res[0])})

final_df = predict_df.join(predict_df.apply(optimized_position, axis=1))

In [36]:
new_result = final_df.sort_values('Search Volume', ascending=False)[final_df["optimized_position"]<10]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [37]:
# Show Keywords where the Model predicts a top 10 Position
new_result

,Position,Keyword,Url,CPC,Competition,Number of Results,Search Volume,optimized_position
82,59.0,barfen,https://www.barf-alarm.de/blog/barf-bei-aelter...,1.43,0.89,394000.0,14800.0,2
1116,NaN,probiotika,NaN,0.69,1.00,1260000.0,12100.0,2
14,25.0,barf,https://www.barf-alarm.de/,1.31,0.95,10300000.0,9900.0,2
1850,NaN,zucchini giftig,NaN,1.03,0.01,94000.0,9900.0,2
2497,NaN,hanföl,NaN,0.78,1.00,328000.0,9900.0,2
1208,NaN,tackenberg,NaN,0.28,0.30,216000.0,6600.0,2
1094,NaN,knochen,NaN,0.35,0.05,16900000.0,6600.0,2
1119,NaN,akut,NaN,0.00,0.00,21300000.0,5400.0,2
1108,NaN,algen,NaN,0.64,0.25,5060000.0,5400.0,2
3637,NaN,barf shop,NaN,1.75,0.92,8180000.0,5400.0,2


In [38]:
new_result.to_csv("result.csv")